# Tokenization

Tokenization is a task of converting input text into individual tokens that a language model can process. We use the text from [The Verdict](https://en.wikisource.org/wiki/The_Verdict) to train out tokenizers. 

We start with a simple character level tokenization. Here, we consider each character as a token. So, for an input text, each unique character in the text becomes a separate token. 

In [1]:
import plotly.express as px
from collections import defaultdict

In [2]:
class CharTokenizer:
    def __init__(self, training_data: str = "the quick brown fox jumps over the lazy dog"):
        self.training_data = training_data
        self.fit()

    def fit(self):
        self.vocab = sorted(set(self.training_data))
        self.char_to_id = {char: idx for idx, char in enumerate(self.vocab)}
        self.id_to_char = {idx: char for idx, char in enumerate(self.vocab)}
        self.vocab_size = len(self.vocab)

    def plot_token_distribution(self):
        counter = defaultdict(int)
        for token in self.encode(self.training_data):
            counter[token] += 1
        freq = [counter[self.char_to_id[char]] for char in self.vocab]
        fig = px.bar(x=self.vocab, y=freq, labels={'x': 'Token', 'y': 'Frequency'}, width=800, height=300)
        fig.show()

    def get_compression_ratio(self):
        num_bytes = len(bytes(self.training_data, encoding="utf-8"))
        num_tokens = len(self.encode(self.training_data))
        return num_bytes / num_tokens

    def encode(self, text: str):
        return [self.char_to_id[char] if char in self.char_to_id else '<unk>' for char in text]

    def decode(self, ids: list):
        return ''.join([self.id_to_char[id_] if type(id_) is int else '<unk>' for id_ in ids])

tk = CharTokenizer()
print(tk.vocab_size, tk.vocab)
print(tk.encode("hi, how are you?"))
print(tk.decode(tk.encode("hi, how are you?")))
print('Compression ratio', tk.get_compression_ratio())
tk.plot_token_distribution()

with open("./data/the-verdict.txt", "r") as f:
    data = f.read()
tk = CharTokenizer(data)
print(tk.vocab_size, tk.vocab)
print(tk.encode("hi, how are you?"))
print(tk.decode(tk.encode("hi, how are you?")))
print('Compression ratio', tk.get_compression_ratio())
tk.plot_token_distribution()

27 [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
[8, 9, '<unk>', 0, 8, 15, 23, 0, 1, 18, 5, 0, 25, 15, 21, '<unk>']
hi<unk> how are you<unk>
Compression ratio 1.0


62 ['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
[43, 44, 7, 1, 43, 50, 58, 1, 36, 53, 40, 1, 60, 50, 56, 12]
hi, how are you?
Compression ratio 1.0


As you can see here vocab sizes are quite small (<100) and the token distribution is quite skewed. This might not be bad for small language models, but if we want to train language models with more and more text, a small vocab size would lead to a very high sequence length which may not be ideal. On the other hand, a very high vocab size can hit curse of dimensionality. 

We thus use a term called compression ratio, which you see above. It is an expected ratio of number of tokens we generate as a sequence for an input text with respect to the byte count of the input. A higher compression ratio shows that our training data would not be as long for us to train with.

Hence, we need a controlled measure for tokenization. One of the ways to do this is use Byte-Pair-Encoding (BPE). Basically, you start with a byte level tokenizer which maps each byte to a new token, hence giving a vocab size of 256. To increase the vocab size in a controlled manner that leads to the highest uplift in the compression ratio, we find the pair of two bytes in the training data that appear the most frequently and allocate a new token for them. We continue this until we get a high enough vocab size and a decent compression ratio. 


References: [Stanford CS336 lecture 1](https://www.youtube.com/watch?v=SQ3fZ1sAqXI&list=PLoROMvodv4rOY23Y0BoGoBGgQ1zmU_MT_), [Andrey Karpathy's video](https://www.youtube.com/watch?v=zduSFxRajkE), [TikTokenizer Visualization](https://tiktokenizer.vercel.app/).

In [6]:
from rich.progress import track

class BPETokenizer(CharTokenizer):
    def __init__(self, training_data: str = "the quick brown fox jumps over the lazy dog", num_merges: int = 10, verbose=False):
        self.num_merges = num_merges
        self.verbose = verbose
        super().__init__(training_data)

    def fit(self):
        self.training_data_bytes = list(self.training_data.encode('utf-8'))
        self.vocab_size = 256
        self.vocab = [chr(i) for i in range(self.vocab_size)]
        self.char_to_id = {char: idx for idx, char in enumerate(self.vocab)}
        self.id_to_char = {idx: char for idx, char in enumerate(self.vocab)}
        self.compression_ratios = [1.0] 

        for _ in track(range(self.num_merges), description="Training BPE..."):
            pair_counts = defaultdict(int)
            
            # count pairs
            for i in range(len(self.training_data_bytes) - 1):
                pair = (self.training_data_bytes[i], self.training_data_bytes[i + 1])
                pair_counts[pair] += 1

            max_pair = max(pair_counts, key=pair_counts.get)            
            self.vocab.append(''.join([self.id_to_char[max_pair[0]], self.id_to_char[max_pair[1]]]))
            if self.verbose:
                print('Training data so far', [self.id_to_char[b] for b in self.training_data_bytes[:50]])
                print('Top 10 pairs', [(self.id_to_char[c1], self.id_to_char[c2], count) for ((c1, c2), count) in [sorted(pair_counts.items(), key=lambda x: x[1], reverse=True)[:10]][0]])
                print(f'New token "{self.vocab[-1]}"')
            self.vocab_size += 1
            self.char_to_id[self.vocab[-1]] = self.vocab_size - 1
            self.id_to_char[self.vocab_size - 1] = self.vocab[-1]

            # merge pair in training data
            new_data = []
            i = 0
            while i < len(self.training_data_bytes):
                if i < len(self.training_data_bytes) - 1 and (self.training_data_bytes[i], self.training_data_bytes[i + 1]) == max_pair:
                    new_data.append(self.vocab_size - 1)
                    i += 2
                else:
                    new_data.append(self.training_data_bytes[i])
                    i += 1
            self.training_data_bytes = new_data

            # calculate compression ratio
            self.compression_ratios.append(self.get_compression_ratio())

    def plot_compression_ratios(self):
        fig = px.line(x=list(range(len(self.compression_ratios))), y=self.compression_ratios, labels={'x': 'Number of Merges', 'y': 'Compression Ratio'}, width=800, height=300)
        fig.show()

    def encode(self, text: str):
        tokens = []
        i = 0
        while i < len(text):
            match = None
            for j in range(self.vocab_size - 1, -1, -1):
                token = self.id_to_char[j]
                if text.startswith(token, i):
                    match = token
                    break
            if match:
                tokens.append(self.char_to_id[match])
                i += len(match)
            else:
                tokens.append('<unk>')
                i += 1
        return tokens

    def decode(self, ids: list):
        return ''.join([self.id_to_char[id_] if type(id_) is int else '<unk>' for id_ in ids])

tk = BPETokenizer(num_merges=2, verbose=True)

Output()

Training data so far ['t', 'h', 'e', ' ', 'q', 'u', 'i', 'c', 'k', ' ', 'b', 'r', 'o', 'w', 'n', ' ', 'f', 'o', 
'x', ' ', 'j', 'u', 'm', 'p', 's', ' ', 'o', 'v', 'e', 'r', ' ', 't', 'h', 'e', ' ', 'l', 'a', 'z', 'y', ' ', 'd', 
'o', 'g']

Top 10 pairs [('t', 'h', 2), ('h', 'e', 2), ('e', ' ', 2), (' ', 'q', 1), ('q', 'u', 1), ('u', 'i', 1), ('i', 'c', 
1), ('c', 'k', 1), ('k', ' ', 1), (' ', 'b', 1)]

New token "th"

Training data so far ['th', 'e', ' ', 'q', 'u', 'i', 'c', 'k', ' ', 'b', 'r', 'o', 'w', 'n', ' ', 'f', 'o', 'x', ' 
', 'j', 'u', 'm', 'p', 's', ' ', 'o', 'v', 'e', 'r', ' ', 'th', 'e', ' ', 'l', 'a', 'z', 'y', ' ', 'd', 'o', 'g']

Top 10 pairs [('th', 'e', 2), ('e', ' ', 2), (' ', 'q', 1), ('q', 'u', 1), ('u', 'i', 1), ('i', 'c', 1), ('c', 'k',
1), ('k', ' ', 1), (' ', 'b', 1), ('b', 'r', 1)]

New token "the"

Here you see for the training data "the quick brown fox jumps over the lazy dog", in the first iteration, we find the most common token pair is **th**, hence the training data replaces all subsequent occurences of 't' and 'h' to 'th'. Subsequently, we get **the** as the next token and so on. 

Let's now train it on the Verdict data.

In [ ]:
with open("./data/the-verdict.txt", "r") as f:
    data = f.read()
tk = BPETokenizer(data, num_merges=100)
print(tk.vocab_size, tk.vocab)
print(tk.encode("hi, how are you?"))
print(tk.decode(tk.encode("hi, how are you?")))
print('Compression ratio', tk.get_compression_ratio())
print('Added tokens ', tk.vocab[256:])
tk.plot_token_distribution()

Output()

Nice! So the BPE tokenizer create tokens for 'e ', ' t', 'd ', 't ', 'in' and so on, and we see they appear quite frequently too, giving us a compression ratio of 1.66. 

Another interesting aspect of BPE tokenizer is that as we increase the number of merges, we get diminishing returns for the compression ratio. 

In [ ]:
tk.plot_compression_ratios()